In [315]:
import pandas as pd

df = pd.read_csv('nfl-cleaned.csv')

In [316]:
df.sort_values('Date')
df.drop(columns=['Unnamed: 0'])
df = df.reset_index()

We must now compute the expanding averages over previous games in order to train
and test our model. $\newline$
This is because we can only use statistics from previous 
games to predict our incoming test game. $\newline$
We use expanding instead of rolling averages because we want the expanding mean
over all previous games, not just a fixed sliding window.

In [317]:
# Compute the expanding averages over previous games. 

numeric_columns = ['Home',
				'Team1Pts',    
				'Team2Pts',    
				'Team1PtDiff',
				'Team2PtDiff', 
				'Team1TM',    
				'Team2TM',     
				'Team1Rating', 
				'Team2Rating', 
				'Team1Sks',    
				'Team2Sks',    
				'Team1SkYds',  
				'Team2SkYds',  
				'Team1RushAtt',
				'Team2RushAtt',
				'Team1RushYds',
				'Team2RushYds',
				'Team1RYM',    
				'Team2RYM',    
				'Team1PYM',    
				'Team2PYM',    
				'Team1YM',     
				'Team2YM']

for column in numeric_columns:
	avg_col_name = column + '_avg'
	df[avg_col_name] = (
		df.groupby('Team1', group_keys=False)[column]
		.apply(lambda group: group.expanding().mean().shift(1))
		.reset_index(drop=True)
	)

In [318]:
df[df['Team1'] == 'DAL'].head()

,index,Unnamed: 0,ID,Team1Won,Season,Date,Team1,Team2,Home,Team1Pts,...,Team1RushAtt_avg,Team2RushAtt_avg,Team1RushYds_avg,Team2RushYds_avg,Team1RYM_avg,Team2RYM_avg,Team1PYM_avg,Team2PYM_avg,Team1YM_avg,Team2YM_avg
6,6,6,1459,False,2022,2022-09-11,DAL,TAM,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,47,47,1383,True,2022,2022-09-26,DAL,NYG,0,23,...,18.000000,33.000000,71.0,152.000000,-81.000000,81.000000,-22.000000,22.000000,-103.0,103.0
53,53,53,1359,True,2022,2022-10-02,DAL,WAS,1,25,...,24.000000,29.000000,123.5,159.500000,-36.000000,36.000000,12.000000,-12.000000,-24.0,24.0
74,74,74,1341,True,2022,2022-10-09,DAL,LAR,0,22,...,25.666667,28.333333,103.0,153.666667,-50.666667,50.666667,28.666667,-28.666667,-22.0,22.0
88,88,88,1309,False,2022,2022-10-16,DAL,PHI,0,17,...,27.750000,25.000000,118.0,124.750000,-6.750000,6.750000,-30.750000,30.750000,-37.5,37.5


Let's hold out a test set and set it aside for later use.

In [319]:
import numpy as np

train_set, test_set = np.split(df, [int(0.8 * len(df))])

C:\Users\13462\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Let's do cross validation and training!

In [320]:
# Prepare train and test sets for each fold of TimeSeriesSplit

pre_game_cols = ['Team1', 'Team2', 'Home']
train_post_game_cols = ['Team1Pts', 'Team2Pts', 'Team1RushYds', 'Team2RushYds', 'Team1SkYds', 'Team2SkYds',
                  'Team1Sks', 'Team2Sks', 'Team1RushAtt', 'Team2RushAtt', 'Team1RYM', 'Team2RYM', 
                  'Team1PYM', 'Team2PYM', 'Team1YM', 'Team2YM', 'Team1Rating', 'Team2Rating']

test_post_game_cols = [col + '_avg' for col in train_post_game_cols]

outcome_col = 'Team1Won'

def prep_data_for_fold(fold, df):
  # Split data into training and testing based on the fold
  train_indices, test_indices = fold
  train_data = df.iloc[train_indices]
  test_data = df.iloc[test_indices]

  # Extract features that will be trained and tested on
  X_train = train_data[train_post_game_cols]
  X_test = test_data[test_post_game_cols]
  
  # Class labels from fold split
  y_train = train_data[outcome_col]
  y_test = test_data[outcome_col]
  
  return X_train, X_test, y_train, y_test

In [325]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# TODO: PCA???

# Nested cross-validation loop that computes how well a model does.
# Return an accuracy score averaged over all folds of the TimeSeriesSplit.

def get_model_accuracy(model, params, df):

  tscv = TimeSeriesSplit(n_splits=5)

  accuracies = []

  # Outer loop: find average accuracy over all time series splits

  for train_indices, test_indices in tscv.split(df):

    # Prepare the data for this fold
    X_train, X_test, y_train, y_test = prep_data_for_fold((train_indices, test_indices), df)

    # Rename the <col>_avg columns to just <col> so GridSearchCV doesn't complain
    X_test = X_test.rename(columns=lambda x: x[:-4] if x.endswith('_avg') else x)

    # Scale the data (fit scaler on training data and transform both train and test)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Inner loop: find the best hyperparameters for this split
    grid_search = GridSearchCV(estimator=model, param_grid=params, cv=tscv)
    grid_search.fit(X_train, y_train)

    # Get the best model from grid search
    best_model = grid_search.best_estimator_

    # Analyze how well model does by comparing its predictions to actual class labels
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
  
  print(best_model)

  # Return the average accuracy across all outer folds
  return sum(accuracies) / len(accuracies)

In [327]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

models_dict = {}

dtc = DecisionTreeClassifier()
dtc_params = {
  'max_depth': [5, 10, 15, 20],
  'max_features': [5, 10, 15],
  'min_samples_leaf': [5, 10, 15, 20]
}

# lr = LogisticRegression()
# lr_params = {
#   'penalty':['l1','l2','elasticnet','none'],
#   'C' : np.logspace(-4,4,20),
#   'solver': ['lbfgs','newton-cg','liblinear','sag','saga'],
#   'max_iter': [100,1000,2500,5000]
# }

rfc = RandomForestClassifier()
rfc_params = {
  'n_estimators': [50, 100, 200],
  'max_depth': [None, 10, 20],
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 4]
}

gbc = GradientBoostingClassifier()
gbc_params = {
  'n_estimators': [50, 100, 200],
  'learning_rate': [0.01, 0.1, 0.2],
  'max_depth': [3, 5, 7],
  'subsample': [0.8, 1.0]
}

knn = KNeighborsClassifier()
knn_params = {
  'n_neighbors': [3, 5, 10],
  'weights': ['uniform', 'distance'],
  'p': [1, 2]  # 1 = Manhattan distance, 2 = Euclidean distance
}

svc = SVC()
svc_params = {
  'C': [0.1, 1, 10],
  'kernel': ['linear', 'rbf', 'poly'],
  'gamma': ['scale', 'auto'],
  # 'degree': [2, 3, 4]  # Only for 'poly' kernel
}

models_dict[dtc] = dtc_params
# models_dict[lr] = lr_params
models_dict[rfc] = rfc_params
models_dict[gbc] = gbc_params
models_dict[knn] = knn_params
models_dict[svc] = svc_params

for model in models_dict.keys():
  score = get_model_accuracy(model, models_dict[model], df)
  print(score)

# TODO: Handle <col>_avg NaN for each team's first game of the season
# LogisticRegression() does not accept missing values encoded as NaN natively.

DecisionTreeClassifier(max_depth=20, max_features=15, min_samples_leaf=5)
0.5934959349593496


KeyboardInterrupt: 